Explore [geo pandas mapping example](https://geopandas.org/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py) with novel corona virus data from [John's Hopkins via Kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset#time_series_covid_19_confirmed_US.csv).

You need to manually download the data because you need a (free) Kaggle account. Download the Kaggle Covid data set and unpack it into the data/ directory containing this notebook.

Make sure to have [geopandas](https://geopandas.org/install.html) installed for base functionality.  The [descartes](https://pypi.python.org/pypi/descartes) package is needed for plotting the map.

!conda install -y geopandas

!conda install -y descartes

Set up the library imports

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [ ]:
rona_df = pd.read_csv("data/time_series_covid_19_confirmed_US.csv")

In [ ]:
rona_df.head()

We want to limit our view to the continental United States.  It would be great to have a [bounding box to select just the Latitude and Longitude of interest](https://wiki.openstreetmap.org/wiki/Bounding_Box) but a quick search didn't turn up any numbers to use.  We can get our info by just dropping non-states in the first six rows who have a UID less than 1000 and Alaska and Hawaii from the states. Syntax help from https://stackoverflow.com/a/17071908/8928529.

We can get just state level data for a less granular map by choose the entries with Lat = 0 but this is more complicated to plot since we lose position info. Go ahead and create the data set but ignore it for now. 

In [ ]:
rona_cus_df = rona_df.loc[(rona_df['UID']>1000) &
                          (rona_df['Lat'] > 0) &
                          (~rona_df['Province_State'].isin(['Alaska', 'Hawaii']))
                         ]

In [ ]:
rona_state_df = rona_df.loc[(rona_df['UID']>1000) & 
                            (rona_df['Lat'] == 0) &
                            (~rona_df['Province_State'].isin(['Alaska', 'Hawaii'])) &
                            (~rona_df['Admin2'].isin(['Unassigned', '']))
                           ]

In [ ]:
rona_cus_df

create teh geodata frame for plotting

In [ ]:
gdf = geopandas.GeoDataFrame(rona_cus_df,
                             geometry=geopandas.points_from_xy(rona_cus_df.Long_, rona_cus_df.Lat))


In [ ]:
gdf

Now plot the location data on the US contentintal map. (This requires the [descartes](https://pypi.python.org/pypi/descartes) module installed above.)

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# We restrict to United States
ax = world[world.continent == 'United States'].plot(
    color='white', edgecolor='black')

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color='black', marker='s', markersize=10)

plt.axis("off")
# note figsize causes the png to be empty, not sure how thes value interact, use dpi for now
# plt.figure(figsize=(10,10))

# add .savefig() to write the image as png
# also note that the output appears to be grayscale since the square edges are soft
# would be good to have simple black and white ouput, zeros and ones.
plt.savefig("murica.png", dpi=100)
plt.show()